In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Row-Column Transformations

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


Sebuah pertanyaan yang mungkin ingin kita tanyakan adalah: berapa rasio laki-laki dan perempuan pada hari yang berbeda dalam seminggu?

Untuk melakukan ini kita mungkin mulai dengan groupby:

In [3]:
tips_gb = tips.groupby(['day', 'sex']).agg({'size': 'sum'})
tips_gb

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

Jadi kita menuju ke suatu tempat, tetapi agak sulit untuk mengetahui jumlah pengunjung pria dan wanita dengan melihatnya, dan Anda mungkin ingin melakukan lebih banyak operasi kolom untuk membandingkan pengunjung pria dan wanita.

Jadi yang mungkin ingin Anda lakukan adalah mengambil nilai di kolom jenis kelamin dan membuatnya menjadi kolom. Di sinilah peran unstacking!

## Unstack

In [4]:
tips_us = tips_gb.unstack()
tips_us

size       
sex  Male Female
day             
Thur   73     79
Fri    21     19
Sat   156     63
Sun   163     53

Perhatikan pada dasarnya, kita memindahkan indeks ke kolom!

In [5]:
# Anda bisa melakukan hal yang sama dengan hari dalam seminggu
tips_gb.unstack(0)

size              
day    Thur Fri  Sat  Sun
sex                      
Male     73  21  156  163
Female   79  19   63   53

Masalahnya adalah sekarang kita memiliki objek baru yang aneh ini sebagai kolom:

In [6]:
tips_us.columns

MultiIndex([('size',   'Male'),
            ('size', 'Female')],
           names=[None, 'sex'])

Dan sementara Anda dapat melakukan banyak hal dengannya:

In [7]:
tips_us[[('size', 'Male')]]

,size
sex,Male
day,
Thur,73
Fri,21
Sat,156
Sun,163


In [8]:
tips_us_copy = tips_us.copy()

tips_us_copy.columns = ['__'.join(col).strip() for col in tips_us.columns.values]

In [9]:
tips_us_copy

,size__Male,size__Female
day,,
Thur,73,79
Fri,21,19
Sat,156,63
Sun,163,53


Anda tentu saja dapat mengulangi operasi itu sebanyak yang Anda butuhkan untuk mendapatkan perincian kolom yang diinginkan.

Tapi sekarang mari kita coba operasi sebaliknya. Ini berguna jika seseorang memberi Anda data dalam bentuk pivot.

## Stack

In [10]:
tips_us.stack()

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

Sekali lagi Anda dapat menghapus salah satu indeks kolom:

In [11]:
tips_us.stack(0)

,sex,Male,Female
day,,,
Thur,size,73,79
Fri,size,21,19
Sat,size,156,63
Sun,size,163,53


## What about Melting and Pivoting?

Itu saja dalam stack dan unstack. Apa pun yang dapat Anda lakukan dengan melt dan pivot dapat dilakukan dengan stack dan unstack. Mari kita lakukan satu contoh dari pandas:

In [12]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                        'last': ['Doe', 'Bo'],
                        'height': [5.5, 6.0],
                        'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [13]:
# melt akan melakukan stacking pada satu operasi
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


Untuk melakukan ini dengan stack kita hanya perlu melakukannya dalam dua langkah:

In [14]:
cheese.set_index(['first', 'last'], inplace=True)
cheese.stack().reset_index()

,first,last,level_2,0
0,John,Doe,height,5.5
1,John,Doe,weight,130.0
2,Mary,Bo,height,6.0
3,Mary,Bo,weight,150.0


## Dummy Variables

Ada satu cara terakhir untuk mengubah nilai dalam kolom menjadi header, dan ini disebut membuat variabel dummy.

Membuat variabel tiruan akan mengambil semua nilai berbeda `k` dalam satu kolom dan membuat kolom `k` darinya.

Mari kita lihat contoh di bawah ini:

In [15]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [16]:
pd.get_dummies(tips.head(), columns=['sex'])

,total_bill,tip,smoker,day,time,size,sex_Male,sex_Female
0,16.99,1.01,No,Sun,Dinner,2,0,1
1,10.34,1.66,No,Sun,Dinner,3,1,0
2,21.01,3.50,No,Sun,Dinner,3,1,0
3,23.68,3.31,No,Sun,Dinner,2,1,0
4,24.59,3.61,No,Sun,Dinner,4,0,1


Perhatikan kolom sex dibagi menjadi kolom sex_Male dan sex_Female. Ketika jenis kelaminnya perempuan, sex_Female adalah 1 dan 0 sebaliknya. Begitu pula untuk kolom sex_Male.

Ini bisa sangat berguna untuk model ML dan melakukan beberapa jenis analisis.